In [1]:
from models.GenerativeAdversarialNetwork import GenerativeAdversarialNetwork

Using TensorFlow backend.


In [2]:
image_dim = (28, 28, 1)
latent_dim = (100, )

In [19]:
generator_initial_dim = (7, 7, 8)
generator_activation = 'relu'
discriminator_activation = 'relu'
use_batch_norm = True
discriminator_dense_dim = 32
generator_convolutional_params = [
    {'strides': (1, 1), 'filters': 32, 'kernel_size': (5, 5), 'upsample': 1, },
    {'strides': (1, 1), 'filters': 32, 'kernel_size': (5, 5), 'upsample': 2, },
    {'strides': (1, 1), 'filters': 32, 'kernel_size': (5, 5), 'upsample': 1, },
    {'strides': (1, 1), 'filters': 32, 'kernel_size': (5, 5), 'upsample': 2, },
    {'strides': (1, 1), 'filters': 1, 'kernel_size': (5, 5), 'upsample': 1, },
    ]
discriminator_convolutional_params = [
    {'strides': (1, 1), 'filters': 32, 'kernel_size': (3, 3),},
    {'strides': (2, 2), 'filters': 32, 'kernel_size': (3, 3),},
    {'strides': (1, 1), 'filters': 32, 'kernel_size': (3, 3),},
    {'strides': (2, 2), 'filters': 32, 'kernel_size': (3, 3),},
    {'strides': (1, 1), 'filters': 32, 'kernel_size': (3, 3),},
    ]

In [20]:
gan = GenerativeAdversarialNetwork(
    image_dim=image_dim,
    latent_dim=latent_dim,
    generator_initial_dim=generator_initial_dim,
    discriminator_dense_dim=discriminator_dense_dim,
    generator_activation=generator_activation,
    discriminator_activation=discriminator_activation,
    generator_convolutional_params=generator_convolutional_params,
    discriminator_convolutional_params=discriminator_convolutional_params,
    use_batch_norm=use_batch_norm,
    )

In [21]:
gan.generator_model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generator_input (InputLayer) (None, 100)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 392)               39592     
_________________________________________________________________
reshape_5 (Reshape)          (None, 7, 7, 8)           0         
_________________________________________________________________
batch_normalization_34 (Batc (None, 7, 7, 8)           32        
_________________________________________________________________
activation_38 (Activation)   (None, 7, 7, 8)           0         
_________________________________________________________________
generator_conv2d_0 (Conv2D)  (None, 7, 7, 32)          6432      
_________________________________________________________________
batch_normalization_35 (Batc (None, 7, 7, 32)          128 

In [22]:
gan.discriminator_model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
discriminator_input (InputLa (None, 28, 28, 1)         0         
_________________________________________________________________
discriminator_conv2d_0 (Conv (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_39 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
activation_43 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
discriminator_conv2d_1 (Conv (None, 14, 14, 32)        9248      
_________________________________________________________________
batch_normalization_40 (Batc (None, 14, 14, 32)        128       
_________________________________________________________________
activation_44 (Activation)   (None, 14, 14, 32)        0  